# Daily 1km data -> Daily 0.05 deg

In [ ]:
cd _1_daily/

In [ ]:
%run ClimateMarble_with_fortran_SW.py

### &Output saturation occurences

This is to check the occurences of MOD021KM Scaled_Integers equals to 65528. Although 65528 results from a bunch of issues regarding high-res sample aggregation, the majority is from the signal saturation for VIS bands (1--7). 

### && by year

In [ ]:
from my_module import h5py, SD, np, os, tqdm



def count_saturation(mod_file):
    mod_daily = h5py.File(mod_file, 'r')
    
    count1 = 0
    count3 = 0
    count4 = 0
    for item in mod_daily.iteritems():
        item = str(item[0])
        tmp_b1 = mod_daily['{}/Scaled_Integers'.format(item)][0]
        tmp_b3 = mod_daily['{}/Scaled_Integers'.format(item)][2]
        tmp_b4 = mod_daily['{}/Scaled_Integers'.format(item)][3]
        count1 += len(np.where(tmp_b1.ravel()==65528)[0])
        count3 += len(np.where(tmp_b3.ravel()==65528)[0])
        count4 += len(np.where(tmp_b4.ravel()==65528)[0])
        
    return count1, count3, count4



iyr = 2000
mod_folder = '/u/sciteam/smzyz/scratch/data/MODIS/MOD02_VIS_daily/{}'.format(iyr)
mod_files = [os.path.join(mod_folder, ifile) for ifile in os.listdir(mod_folder)]

cnt1 = 0
cnt3 = 0
cnt4 = 0
for ifile in mod_files:
    count1, count3, count4 = count_saturation(ifile)
    cnt1 += count1/10000.
    cnt3 += count3/10000.
    cnt4 += count4/10000.
    
    print ifile, cnt1, cnt3, cnt4

### *Output VIS composite images

This serves the visual check for the produced 0.05 deg MODIS VIS composite data.

### ** daily results

In [ ]:


from my_module import np, h5py, plt, os, toimage, tqdm



for iyr in range(2003, 2016):
    folder = "/u/sciteam/smzyz/scratch/results/MODIS_ClimateMarble_005deg/daily/vis/{}".format(iyr)
    tst_files = [os.path.join(folder, ifile) for ifile in os.listdir(folder) if ifile.startswith('A')]

    for tst_file in tqdm(tst_files):
        with h5py.File(tst_file, 'r') as h5f:
            num = h5f['radiance_num'][:]
            solar = h5f['insolation_sum'][:] / num
            rad = h5f['radiance_sum'][:] / num
        #     rad = h5f['monthly_radiance'][:]
        #     solar = h5f['monthly_insolation'][:]

        ref = rad / solar

        r = ref[:, :, 0]
        np.place(r, r>1., 1)
        r = np.nan_to_num(r)

        g = ref[:, :, 3]
        np.place(g, g>1., 1)
        g = np.nan_to_num(g)

        b = ref[:, :, 2]
        np.place(b, b>1., 1)
        b = np.nan_to_num(b)

        toimage(np.dstack([r, g, b])).save('/u/sciteam/smzyz/scratch/results/MODIS_ClimateMarble_005deg/daily/vis_img/{}.png'.format(tst_file[-10:-3]))

 52%|█████▏    | 189/365 [2:43:40<2:32:24, 51.96s/it]

# Daily mean -> Monthly mean

2018.05.23    --    I don't think it is worthwhile writing an individual script to do the process.  
What I learnt from this excercise is 1) Using python build-in matrix operator is 5 times faster than f2py generated fortran code; 2) 

In [25]:


from my_module import np, h5py, os, tqdm
from my_module.data.comm import save_data_hdf5



def daily2monthly_mean(icat, iyr, imon):
    doy_normal = [1, 1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335, 366]
    doy_leap = [1, 1, 32, 61, 92, 122, 153, 183, 214, 245, 275, 306, 336, 367]
    
    data_folder = "/u/sciteam/smzyz/scratch/results/MODIS_ClimateMarble_005deg/daily/{}/".format(icat)
    data_files = os.listdir(data_folder)
    output_dir = "/u/sciteam/smzyz/scratch/results/MODIS_ClimateMarble_005deg/monthly/"
    output_file = os.path.join(output_dir, "{}_{}_{}.h5".format(icat, iyr, str(imon).zfill(2)))
    
    
    if iyr in [2000, 2004, 2008, 2012]:
        doy = doy_leap
    else:
        doy = doy_normal
    
    
    print ">> work on {}.{}".format(iyr, imon)
    if icat in ['vis']:
        radiance_all = np.zeros((3600, 7200, 7))
        insolation_all = np.zeros((3600, 7200, 7))
        num_all = np.zeros((3600, 7200, 7))
    else:
        print ">> err: only supports visible bands right now."
        return
        
        
    doy_0 = doy[imon]
    doy_1 = doy[imon+1]
    
    
    for ifile in data_files:
        if ifile.endswith('.h5') == False:
            continue
        
        iyear  = int(ifile.split('.')[0][1:5])
        iday = int(ifile.split('.')[0][-3:]) # Example: ifile --> A2000265.h5       
        if (iday in range(doy_0, doy_1)) & (iyr == iyear):
            print ifile, iday 
            ifile = os.path.join(data_folder, ifile)
            
            try:
                with h5py.File(ifile, 'r') as h5f:
                    solar = h5f['insolation_sum'][:]
                    rad = h5f['radiance_sum'][:]
                    num = h5f['radiance_num'][:]
                
                
                insolation_all = insolation_all + solar
                radiance_all = radiance_all + rad
                num_all = num_all + num
            except Exception as err:
                print ">> err: {}".format(err)
                continue

                
    mean_rad = np.array(radiance_all / num_all)
    mean_sol = np.array(insolation_all / num_all)
    
    save_data_hdf5(output_file, 'monthly_radiance', mean_rad)
    save_data_hdf5(output_file, 'monthly_insolation', mean_sol)
    


In [33]:
for iyr in range(2001, 2003):
    for imon in range(1, 13):
        daily2monthly_mean('vis', iyr, imon)

>> work on 2001.1
A2001005.h5 5
A2001021.h5 21
A2001012.h5 12
A2001014.h5 14
A2001007.h5 7
A2001009.h5 9
A2001011.h5 11
A2001030.h5 30
A2001006.h5 6
A2001001.h5 1
A2001022.h5 22
A2001004.h5 4
A2001019.h5 19
A2001018.h5 18
A2001028.h5 28
A2001024.h5 24
A2001029.h5 29
A2001003.h5 3
A2001027.h5 27
A2001026.h5 26
A2001016.h5 16
A2001023.h5 23
A2001020.h5 20
A2001008.h5 8
A2001002.h5 2
A2001010.h5 10
A2001015.h5 15
A2001031.h5 31
A2001025.h5 25
A2001017.h5 17
A2001013.h5 13
>> work on 2001.2
A2001035.h5 35
A2001046.h5 46
A2001042.h5 42
A2001032.h5 32
A2001048.h5 48
A2001051.h5 51
A2001059.h5 59
A2001043.h5 43
A2001049.h5 49
A2001033.h5 33
A2001036.h5 36
A2001058.h5 58
A2001055.h5 55
A2001038.h5 38
A2001045.h5 45
A2001053.h5 53
A2001047.h5 47
A2001041.h5 41
A2001037.h5 37
A2001039.h5 39
A2001044.h5 44
A2001052.h5 52
A2001050.h5 50
A2001054.h5 54
A2001056.h5 56
A2001040.h5 40
A2001034.h5 34
A2001057.h5 57
>> work on 2001.3
A2001084.h5 84
A2001088.h5 88
A2001086.h5 86
A2001081.h5 81
A2001067.h